In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
if (code_show){
$('div.input').hide();
} else {
$('div.input').show();
}
code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="View/Hide Raw Code"></form>''')

# Converting the GenBank File to SBOL
1. Upload the GenBank file or place in the same folder.
2. Enter the file name and the URI prefix (something like https://bu.edu/ben) that you want to use.
3. Enter a version (optional).
4. Hit send and it will create an SBOL file with the same name.

In [2]:
from sbol import *
import ipywidgets as widgets
from IPython.display import Javascript, display, clear_output
import requests
import os
import os.path
from ipyupload import FileUpload

upload = FileUpload(
    accept = '.gb',
    multiple = True,
    disabled = False,
    style_button = '',
    compress_level = 0
)

URIPrefix = widgets.Text(
    value = '',
    placeholder = 'Enter URI Prefix',
    description = 'URI Prefix:',
    disabled = False
)

version = widgets.Text(
    value = '',
    placeholder = 'Enter Version (optional)',
    description = 'Version:',
    disabled = False
)

sendButton = widgets.Button(
    description = 'Send',
    disabled = False,
    button_style = '',
    tooltip = 'Converts Genbank file to SBOL',
    icon = ''
)

out = widgets.Output()

display(upload)
display(URIPrefix)
display(version)
display(sendButton)

doc = Document()
Config.setOption('sbol_compliant_uris', True)
Config.setOption('sbol_typed_uris', False)
originalCDs = []
addedCDs = []

cdDictionary = {}

@out.capture()
def sendRequest(sender):
    global originalCDs, cdDictionary, doc
    clear_output()
    
    doc = Document()
    cdDicitonary = {}
    originalCDs = []
    addedCDs = []
    
    if upload.value == None and URIPrefix.value == '':
        print('You need both GenBank file(s) and URI Prefix!')
        return
    if upload.value == None:
        print('You need GenBank file(s)!')
        return
    if URIPrefix.value == '':
        print('You need a URI Prefix!')
        return
    setHomespace(URIPrefix.value)
    
    files = []
    
    for key in upload.value.keys():
        if key[-3:] == '.gb':
            file = open(key, 'w+')
            file.write(upload.value[key]['content'].decode('utf-8'))
            file.close()
            
            files.append(key)
    
    for gbFileName in files:
        
        gbFile = open(gbFileName).read()

        request = { 'options': {'language' : 'SBOL2',
                                'test_equality': False,
                                'check_uri_compliance': False,
                                'check_completeness': False,
                                'check_best_practices': False,
                                'fail_on_first_error': False,
                                'provide_detailed_stack_trace': False,
                                'subset_uri': '',
                                'uri_prefix': URIPrefix.value,
                                'version': version.value if version.value != '' else '1',
                                'insert_type': False,
                                'main_file_name': 'main file',
                                'diff_file_name': 'comparison file',
                                },
                    'return_file': True,
                    'main_file': gbFile
                  }

        response = requests.post("http://www.async.ece.utah.edu/validate/", json=request)
        xmlResponse = response.json()['result']
    
        sbolFile = open(gbFileName[:-3] + '.xml', 'w+')
        sbolFile.write(xmlResponse)
        sbolFile.close()
        
    
    
        fileNameSBOL = gbFileName[:-3] + '.xml'
        doc.append(fileNameSBOL)
        
        for cd in doc.componentDefinitions:
            exists = False
            for i in originalCDs:
                if i.displayId == cd.displayId:
                    exists = True
            if exists == False:
                originalCDs.append(cd)
                cdDictionary[cd.displayId] = []
#         originalCDs = doc.componentDefinitions
        
    print('Success!')
        
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))

sendButton.on_click(sendRequest)
display(out)

FileUpload(accept='.gb', multiple=True)

Text(value='', description='URI Prefix:', placeholder='Enter URI Prefix')

Text(value='', description='Version:', placeholder='Enter Version (optional)')

Button(description='Send', style=ButtonStyle(), tooltip='Converts Genbank file to SBOL')

Output()

# Reading in the SBOL File
Enter the file name of an SBOL file (.xml or .rdf) and displays the Component Definitions.

## Note that this is only for testing. If you converted the file from GenBank, then it will automatically be read in.

In [11]:
from sbol import *
import ipywidgets as widgets
from IPython.display import Javascript, display, clear_output
import requests
import os
import os.path

fileNameSBOL = widgets.Text(
    value = '',
    placeholder = 'Enter SBOL file name',
    description = 'File Name:',
    disabled = False
)

submit = widgets.Button(
    description = 'Submit',
    disabled = False,
    button_style = '',
    tooltip = 'Reads in the SBOL file',
    icon = ''
)

out = widgets.Output()

# display(fileNameSBOL)
# display(submit)

# doc = Document()
# setHomespace('https://bu.edu/ben')
Config.setOption('sbol_typed_uris', False)
PATH = './'

# originalCDs = []


@out.capture()
def showDoc(sender):
    global originalCDs
    clear_output()
    if fileNameSBOL.value != '':
        if os.path.isfile(PATH + fileNameSBOL.value) and (fileNameSBOL.value[-3:] == 'xml' or fileNameSBOL.value[-3:] == 'rdf'):
            doc.read(fileNameSBOL.value)
            originalCDs = doc.componentDefinitions
            print(doc)
        else:
            print('Invalid file name!')
    else:
        print('You need a file name!')
    
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))


submit.on_click(showDoc)
# display(out)

# Add a Component
Enter the name of the component, the type, and the role (optional).

In [12]:
from sbol import *
import ipywidgets as widgets
from IPython.display import clear_output, Markdown

componentTypes = ['DNA', 'RNA', 'Protein', 'Small Molecule', 'Complex']

componentRoles = ['', 'Miscellaneous', 'Promoter', 'RBS', 'CDS', 'Terminator',
                  'Gene', 'Operator', 'Engineered Gene', 'mRNA', 'Effector']

typesMap = {'DNA':'http://www.biopax.org/release/biopax-level3.owl#DnaRegion',
            'RNA':'http://www.biopax.org/release/biopax-level3.owl#RnaRegion',
            'Protein':'http://www.biopax.org/release/biopax-level3.owl#Protein',
            'Small Molecule':'http://www.biopax.org/release/biopax-level3.owl#SmallMolecule',
            'Complex':'http://www.biopax.org/release/biopax-level3.owl#Complex'}

rolesMap = {'Miscellaneous':'http://identifiers.org/so/SO:0000001',
            'Promoter':'http://identifiers.org/so/SO:0000167',
            'RBS':'http://identifiers.org/so/SO:0000139',
            'CDS':'http://identifiers.org/so/SO:0000316',
            'Terminator':'http://identifiers.org/so/SO:0000141',
            'Gene':'http://identifiers.org/so/SO:0000704',
            'Operator':'http://identifiers.org/so/SO:0000057',
            'Engineered Gene':'http://identifiers.org/so/SO:0000280',
            'mRNA':'http://identifiers.org/so/SO:0000234',
            'Effector':'http://identifiers.org/chebi/CHEBI:35224',
            'Transcription Factor':'http://identifiers.org/go/GO:0003700'}

componentName = widgets.Text(
    value = '',
    placeholder = 'Enter a component name',
    description = 'Name:',
    disabled = False,
)

typeDrop = widgets.Dropdown(
    options = componentTypes,
    description = 'Type:',
    disabled = False
)

roleDrop = widgets.Dropdown(
    options = componentRoles,
    description = 'Role:',
    disabled = False
)

createButton = widgets.Button(
    description = 'Create',
    disabled = False,
    button_style = '',
    tooltip = 'Creates a new Component Definition ',
    icon = ''
)

outBox = widgets.Output()

display(componentName)
display(typeDrop)
display(roleDrop)
display(createButton)
display(outBox)

@outBox.capture()
def createButtonHandler(sender):
    global addedCDs
    clear_output()
    if componentName.value == '':
        print('You need a component name!')
        return
    
    try:
        newComponentDefinition = ComponentDefinition(componentName.value, typesMap[typeDrop.value])
        if roleDrop.value != '':
            newComponentDefinition.roles = rolesMap[roleDrop.value]
    except Exception as e:
        print(e)
        return
    
    try:
        addedCDs.append(newComponentDefinition)
        doc.addComponentDefinition(newComponentDefinition)
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))
        print('Component created!')
    except RuntimeError:
        print('Component name already exists!')

createButton.on_click(createButtonHandler)

Text(value='', description='Name:', placeholder='Enter a component name')

Dropdown(description='Type:', options=('DNA', 'RNA', 'Protein', 'Small Molecule', 'Complex'), value='DNA')

Dropdown(description='Role:', options=('', 'Miscellaneous', 'Promoter', 'RBS', 'CDS', 'Terminator', 'Gene', 'O…

Button(description='Create', style=ButtonStyle(), tooltip='Creates a new Component Definition ')

Output()

# Component Definitions

In [34]:
from sbol import *
import ipywidgets as widgets
from IPython.display import clear_output, Markdown

typeDict = {'http://www.biopax.org/release/biopax-level3.owl#DnaRegion':'DNA',
            'http://www.biopax.org/release/biopax-level3.owl#RnaRegion':'RNA',
            'http://www.biopax.org/release/biopax-level3.owl#Protein':'Protein',
            'http://www.biopax.org/release/biopax-level3.owl#SmallMolecule': 'Small Molecule',
            'http://www.biopax.org/release/biopax-level3.owl#Complex':'Complex'}

roleDict = {'http://identifiers.org/so/SO:0000001':'Miscellaneous', 
            'http://identifiers.org/so/SO:0000167':'Promoter',
            'http://identifiers.org/so/SO:0000139':'RBS',
            'http://identifiers.org/so/SO:0000316':'CDS',
            'http://identifiers.org/so/SO:0000141':'Terminator',
            'http://identifiers.org/so/SO:0000704':'Gene',
            'http://identifiers.org/so/SO:0000057':'Operator',
            'http://identifiers.org/so/SO:0000280':'Engineered Gene',
            'http://identifiers.org/so/SO:0000234':'mRNA',
            'http://identifiers.org/chebi/CHEBI:35224':'Effector',
            'http://identifiers.org/go/GO:0003700':'Transcription Factor'}

def createHTMLString(cd, parentName = None):
    htmlString = ''
    if cd.description is not None:
        htmlString += '<dt><u>Description</u>:</dt>' + cd.description + '</dt>'
    if len(cd.types) is not 0:
        htmlString += '<dt><u>Type(s)</u>:</dt>'
        for typeComponent in cd.types:
            if typeComponent in typeDict:
                htmlString += '<dd>-&nbsp' + typeDict[typeComponent] + '</dd>'
            else:
                htmlString += '<dd>-&nbspOther</dd>' 
    if len(cd.roles) is not 0:
        htmlString += '<dt><u>Role(s)</u>:</dt>'
        for role in cd.roles:
            if role in roleDict:
                htmlString += '<dd>-&nbsp' + roleDict[role] + '</dd>'
            else:
                htmlString += '<dd>-&nbspOther</dd>'            
    if len(cd.components) is not 0:
        htmlString += '<dt><u>Sub-Components</u>:</dt>'
        for component in cd.components:
            if parentName is not None:
                htmlString += '<dd>-&nbsp' + component.name.replace(parentName + '_', '') + '</dd>'
            else:
                htmlString += '<dd>-&nbsp' + component.name + '</dd>'
                
    return htmlString

def createAccordionWidget(components, parentName = None, isDisplayID = False):
    accordion = widgets.Accordion()
    accordionChildren = []
    
    for index, cd in enumerate(components):
        htmlWidget = widgets.HTML()
        accordionChildren.append(htmlWidget)
        
        if isDisplayID == False:
            accordion.set_title(index, cd.displayId)
            htmlString = createHTMLString(cd, parentName)
        else:
            componentDef = doc.getComponentDefinition(cd)
            accordion.set_title(index, componentDef.displayId.replace(parentName + '_', ''))
            htmlString = createHTMLString(componentDef, parentName)
        
        htmlWidget.value = '<d1>' + htmlString + '</d1>'
        
    accordion.children = accordionChildren
    
    return accordion

copyCDs = []

for cd in doc.componentDefinitions:
    num = 0
    name = cd.displayId
    for sa in cd.sequenceAnnotations:
        if sa.name == 'misc_feature':
            sa.name = 'misc_feature' + str(num)
            num = num + 1
            
        if name not in sa.name:
            sa.name = name + '_' + sa.name
            sa.displayId = name + '_' + sa.displayId
            
    copyCDs.append(cd)

for cd in copyCDs:
    currentName = cd.displayId
    if len(cd.components) == 0 and len(cd.sequenceAnnotations) != 0:
        for sa in cd.sequenceAnnotations:
            if sa.component == None:
                createdCD = sa.extract()
                cdDictionary[currentName].append(createdCD.identity)
               
            
mainBox = widgets.VBox()

boxChildren = []

for i, ocd in enumerate(originalCDs):
    boxChildren.append(widgets.HTML('<font size=3><b>' + ocd.displayId + '</b></font>'))
    name = ocd.displayId
    subComponents = cdDictionary[name].copy()
    subComponents.insert(0, ocd.identity)
    
    accordion = createAccordionWidget(subComponents, name, True)
    
    boxChildren.append(accordion)
    boxChildren.append(widgets.HTML('<br>'))
    
if len(addedCDs) != 0:
    boxChildren.append(widgets.HTML('<font size=3><b>Added Components</b></font>'))
    accordion = createAccordionWidget(addedCDs)
    accordion.selected_index = None
    boxChildren.append(accordion)
    
        
mainBox.children = boxChildren
                        
display(mainBox)

# Adding Interactions
1. Select more than one component with `shift + click` or `control (cmd) + click`.  
2. Select the interaction and submit.
3. Select the role that each component plays in the interaction.
4. Press create to create the interaction.

In [35]:
from sbol import *
import ipywidgets as widgets
from IPython.display import clear_output, Markdown

# setHomespace('https://bu.edu/ben')

# modules = []

componentNames = []
mapComponents = {}

for cd in doc.componentDefinitions:
    componentNames.append(cd.displayId)
    mapComponents[cd.displayId] = cd
    
componentNames.sort()
    
typesInteractions = ['Inhibition', 'Stimulation', 'Biochemical Reaction', 
                     'Non-Covalent Binding', 'Degradation', 'Genetic Production', 
                     'Control']
mapInteractions = {'Inhibition':'http://identifiers.org/biomodels.sbo/SBO:0000169', 
                   'Stimulation':'http://identifiers.org/biomodels.sbo/SBO:0000170', 
                   'Biochemical Reaction':'http://identifiers.org/biomodels.sbo/SBO:0000176', 
                     'Non-Covalent Binding':'http://identifiers.org/biomodels.sbo/SBO:0000177', 
                   'Degradation':'http://identifiers.org/biomodels.sbo/SBO:0000179', 
                   'Genetic Production':'http://identifiers.org/biomodels.sbo/SBO:0000589', 
                     'Control':'http://identifiers.org/biomodels.sbo/SBO:0000168'}

typesParticipants = ['Inhibitor', 'Inhibited', 'Stimulator', 'Stimulated', 'Reactant',
                     'Product', 'Promoter', 'Modifier', 'Modified', 'Template']
mapParticipants = {'Inhibitor':'http://identifiers.org/biomodels.sbo/SBO:0000020',
                   'Inhibited':'http://identifiers.org/biomodels.sbo/SBO:0000642',
                   'Stimulator':'http://identifiers.org/biomodels.sbo/SBO:0000459',
                   'Stimulated':'http://identifiers.org/biomodels.sbo/SBO:0000643',
                   'Reactant':'http://identifiers.org/biomodels.sbo/SBO:0000010',
                   'Product':'http://identifiers.org/biomodels.sbo/SBO:0000011',
                   'Promoter':'http://identifiers.org/biomodels.sbo/SBO:0000598',
                   'Modifier':'http://identifiers.org/biomodels.sbo/SBO:0000019',
                   'Modified':'http://identifiers.org/biomodels.sbo/SBO:0000644',
                   'Template':'http://identifiers.org/biomodels.sbo/SBO:0000645'}


participants = widgets.SelectMultiple(
    options = componentNames,
    description = 'Components:',
    rows = len(componentNames),
    disabled = False
)

interaction = widgets.Dropdown(
    options = typesInteractions,
    description = 'Interaction:',
    value = None,
    disabled = False
)

submitButton = widgets.Button(
    description = 'Submit',
    disabled = False,
    button_style = '',
    tooltip = 'Converts ',######## ADJKLAKLSDJL
    icon = ''
)

changeButton = widgets.Button(
    description = 'Change',
    disabled = False,
    button_style = '',
    tooltip = 'Changes interaction',
    icon = ''
)

createInteraction = widgets.Button(
    description = 'Create Interaction',
    disabled = False,
    button_style = '',
    tooltip = 'Creates interaction',
    icon = ''
)

interactionWidgets = widgets.VBox()

outputBox = widgets.Output()

display(participants)
display(interaction)
display(submitButton)
display(outputBox)

@outputBox.capture()
def printMessage(line):
    print(line)

    

module = ModuleDefinition('md0')
foundModule = None
for md in doc.moduleDefinitions:
    if module.identity == md.identity:
        foundModule = md

if foundModule is None:
    doc.addModuleDefinition(module)
else:
    module = foundModule


createdFCs = []
createdInteractions = []
createdMapsTos = []
    
def submitResponder(sender):
    global module, currentInteraction, originalCDs, createdMapsTos
    
    outputBox.clear_output()
    
    if len(participants.value) == 0:
        printMessage('You need to select components!')
        return
    
    items = []
    
    interactionString = interaction.value.replace(' ', '_').replace('-', '_')
    
    for p in participants.value:
        fc = None
        try:
            fc = module.functionalComponents.create(p)
            fc.definition = mapComponents[p].identity
            fc.access = SBOL_ACCESS_PUBLIC
            fc.direction = SBOL_DIRECTION_NONE
            
            createdFCs.append(fc.identity)
        except:
            pass
            
        items.append(widgets.Dropdown(options = typesParticipants, description = p + ':', disabled = False, 
                     style = {'description_width': 'initial'}))
        
        interactionString += '_' + p
        
        
        # Checking to see if parent component exists as a functional component
        for original in originalCDs:
            for c in original.components:
                if c.definition == mapComponents[p].identity:
                    fcP = None
                    try:
                        fcP = module.functionalComponents.create(original.displayId)
                        fcP.definition = original.identity
                        fcP.access = SBOL_ACCESS_PUBLIC
                        fcP.direction = SBOL_DIRECTION_NONE
                    except:
                        for fc in module.functionalComponents:
                            if fc.definition == original.identity:
                                fcP = fc
                                
                    if fc is not None:
                        if fc.definition == c.definition:
                            foundMap = False
                            for mt in fcP.mapsTos:
                                if mt.displayId == str('map_' + fc.displayId):
                                    foundMap = True
                            
                            if foundMap == False:
                                mapsto = fcP.mapsTos.create('map_' + fc.displayId)
                        
                                mapsto.refinement = SBOL_REFINEMENT_USE_LOCAL
                                mapsto.local = fc.identity
                                mapsto.remote = c.identity
                                
                                createdMapsTos.append(mapsto.identity)
    
    
    interactionWidgets.children = items
    
    try:
        interactionModule = module.interactions.create(interactionString)
        interactionModule.types = mapInteractions[interaction.value]
        currentInteraction = interactionModule
        createdInteractions.append(interactionModule.identity)
    except:
        printMessage('Interaction already exists!')
        return
    
    
    participants.disabled = True
    interaction.disabled = True
    submitButton.disabled = True
    
    if changeButton.layout.display == 'none':
        changeButton.layout.display = None
        interactionWidgets.layout.display = None
        createInteraction.layout.display = None
    else:
        display(changeButton)
        display(interactionWidgets)
        display(createInteraction)
    


def changeResponder(sender):
    global createdFCs, createdInteractions, createdMapsTos
    
    interactionWidgets.children = []
    interactionWidgets.hidden = True
    interactionWidgets.layout.display = 'none'
    changeButton.layout.display = 'none'
    createInteraction.layout.display = 'none'
    
    participants.disabled = False
    interaction.disabled = False
    submitButton.disabled = False
    
    for mapsTosURI in createdMapsTos:
        for fc in module.functionalComponents:
            exists = False
            for mt in fc.mapsTos:
                if mapsTosURI == mt.identity:
                    exists = True
            if exists == True:
                fc.mapsTos.remove(mapsTosURI)
    
    for fcURI in createdFCs:
        module.functionalComponents.remove(fcURI)
        
    for iURI in createdInteractions:
        module.interactions.remove(iURI)
        
    createdFCs = []
    createdInteractions = []
    createdMapsTos = []
    
    
def createResponder(sender):
    global createdFCs, createdInteractions, createdMapsTos, module
    alreadyExist = False
    
    
    for index, widget in enumerate(interactionWidgets.children):
        participation = currentInteraction.participations.create(widget.description[:-1])
        participation.roles = mapParticipants[widget.value]
        
        fcIdentity = ''
        for fc in module.functionalComponents:
            if fc.displayId == widget.description[:-1]:
                fcIdentity = fc.identity
        
        participation.participant = fcIdentity
    

    
    interactionWidgets.children = []
    interactionWidgets.hidden = True
    interactionWidgets.layout.display = 'none'
    changeButton.layout.display = 'none'
    createInteraction.layout.display = 'none'
    
    participants.disabled = False
    interaction.disabled = False
    submitButton.disabled = False
    
    participants.value = []
    interaction.value = None
    
    createdFCs = []
    createdInteractions = []
    createdMapsTos = []
    
    if alreadyExist == False:
        printMessage('Interaction created!')
        
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))



submitButton.on_click(submitResponder)
changeButton.on_click(changeResponder)
createInteraction.on_click(createResponder)

SelectMultiple(description='Components:', options=('BBa_J06505', 'BBa_J06505_CDS', 'BBa_J06505_misc_feature0',…

Dropdown(description='Interaction:', options=('Inhibition', 'Stimulation', 'Biochemical Reaction', 'Non-Covale…

Button(description='Submit', style=ButtonStyle(), tooltip='Converts ')

Output()

# Interactions

In [25]:
from sbol import *
import ipywidgets as widgets
from IPython.display import clear_output, Markdown

typesInteractions = ['Inhibition', 'Stimulation', 'Biochemical Reaction', 
                     'Non-Covalent Binding', 'Degradation', 'Genetic Production', 
                     'Control']
interactionMap = {'http://identifiers.org/biomodels.sbo/SBO:0000169':'Inhibition', 
                   'http://identifiers.org/biomodels.sbo/SBO:0000170':'Stimulation', 
                   'http://identifiers.org/biomodels.sbo/SBO:0000176':'Biochemical Reaction', 
                   'http://identifiers.org/biomodels.sbo/SBO:0000177':'Non-Covalent Binding', 
                   'http://identifiers.org/biomodels.sbo/SBO:0000179':'Degradation', 
                   'http://identifiers.org/biomodels.sbo/SBO:0000589':'Genetic Production', 
                   'http://identifiers.org/biomodels.sbo/SBO:0000168':'Control'}

typesParticipants = ['Inhibitor', 'Inhibited', 'Stimulator', 'Stimulated', 'Reactant',
                     'Product', 'Promoter', 'Modifier', 'Modified', 'Template']
participantMap = {'http://identifiers.org/biomodels.sbo/SBO:0000020':'Inhibitor',
                   'http://identifiers.org/biomodels.sbo/SBO:0000642':'Inhibited',
                   'http://identifiers.org/biomodels.sbo/SBO:0000459':'Stimulator',
                   'http://identifiers.org/biomodels.sbo/SBO:0000643':'Stimulated',
                   'http://identifiers.org/biomodels.sbo/SBO:0000010':'Reactant',
                   'http://identifiers.org/biomodels.sbo/SBO:0000011':'Product',
                   'http://identifiers.org/biomodels.sbo/SBO:0000598':'Promoter',
                   'http://identifiers.org/biomodels.sbo/SBO:0000019':'Modifier',
                   'http://identifiers.org/biomodels.sbo/SBO:0000644':'Modified',
                   'http://identifiers.org/biomodels.sbo/SBO:0000645':'Template'}

interactions = doc.moduleDefinitions[0].interactions
numInteractions = len(interactions)
contents = []
for i in range(numInteractions):
    contents.append(widgets.VBox())
    
accordion = widgets.Accordion(children = contents)

for index, inter in enumerate(interactions):
    box = contents[index]
    accordion.set_title(index, inter.displayId)
    childrenList = []
    htmlString = ''
    
    htmlString += '<dt><u>Interaction Type(s)</u>:</dt>'
    for typeI in inter.types:
        htmlString += '<dd>' + interactionMap[typeI] + '</dd>'
    
    htmlString += '<dt><u>Participants</u>:</dt>'
    for participation in inter.participations:
        htmlString += '<dd>-&nbsp' + participation.displayId
        for role in participation.roles:
            htmlString += str('&nbsp(' + participantMap[role] + ')')
        htmlString += '</dd>'

    childrenList.append(widgets.HTML('<d1>' + htmlString + '</d1>'))
    box.children = childrenList
    

display(accordion)



Accordion()

# Upload to SynBioHub
To upload to SynBioHub:
1. Enter your username and password (you need to have an account to upload).
2. Enter a name, description, and version for the Collection.
3. Press Submit.

In [26]:
from sbol import *
import ipywidgets as widgets
from IPython.display import clear_output, Markdown

loginLabel = widgets.HTML('<font size="4"><b>Login to SynBioHub</b></font>')

usernameField = widgets.componentName = widgets.Text(
    value = '',
    placeholder = 'Enter email',
    description = 'Email:',
    disabled = False,
)

passwordField = widgets.Password(description = 'Password:', placeholder = 'Enter password')

loginSubmitButton = widgets.Button(
    description = 'Submit',
    disabled = False,
    button_style = '',
    tooltip = 'Submit annotated Document',
    icon = ''
)

errorBox = widgets.Output()

# Name = Display Id
propertiesLabel = widgets.HTML('<font size="4"><b>Enter Name, Description, and Version</b></font>')

docName = widgets.componentName = widgets.Text(
    value = '',
    placeholder = 'Enter name',
    description = 'Name:',
    disabled = False,
)

docDescription = widgets.componentName = widgets.Text(
    value = '',
    placeholder = 'Enter description',
    description = 'Description:',
    disabled = False,
)

docVersion = widgets.componentName = widgets.Text(
    value = '',
    placeholder = 'Enter version',
    description = 'Version:',
    disabled = False,
)

display(loginLabel)
display(usernameField)
display(passwordField)
display(propertiesLabel)
display(docName)
display(docDescription)
display(docVersion)
display(loginSubmitButton)
display(errorBox)

sbh = PartShop('https://synbiohub.org')



@errorBox.capture()
def loginSBH(sender):
    clear_output()
    if usernameField.value == '':
        print('You need a username!')
        return
    if passwordField.value == '':
        print('You need a password!')
        return
    if docName.value == '':
        print('You need a name for the Collection!')
        return
    if docDescription.value == '':
        print('You need a description for the Collection!')
        return
    if docVersion.value == '':
        print('You need a version for this Collection!')
        return
    
    sbh.login(usernameField.value, passwordField.value)
    
    doc.displayId = docName.value
    doc.name = docName.value
    doc.description = docDescription.value
    doc.version = docVersion.value

    message = sbh.submit(doc)
#     cleanedMessage = message.replace('[','').replace('\"','').replace(']','')
    print(message)


loginSubmitButton.on_click(loginSBH)

HTML(value='<font size="4"><b>Login to SynBioHub</b></font>')

Text(value='', description='Email:', placeholder='Enter email')

Password(description='Password:', placeholder='Enter password')

HTML(value='<font size="4"><b>Enter Name, Description, and Version</b></font>')

Text(value='', description='Name:', placeholder='Enter name')

Text(value='', description='Description:', placeholder='Enter description')

Text(value='', description='Version:', placeholder='Enter version')

Button(description='Submit', style=ButtonStyle(), tooltip='Submit annotated Document')

Output()

In [27]:
# for md in doc.moduleDefinitions:
#     for fc in md.functionalComponents:
#         for i in fc.mapsTos:
#             print(i)


# doc.write("testannotation3.xml")
# doc.validate()
# print(doc.writeString())

# for seq in doc.sequences:
#     print(seq)
# doc.write('hi.xml')


# boxChildren
# for cd in doc.componentDefinitions:
#     for sa in cd.sequenceAnnotations:
#         print(sa.name)
# doc.write('ithinkigoofed.xml')



# print(addedCDs[0].description)